In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install --upgrade nltk
!pip install hdbscan
!pip install dash_bio

In [ ]:
import pandas as pd
import numpy as np
from plotly import graph_objs as go
import plotly.express as px
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.manifold import TSNE
import nltk
import matplotlib.pyplot as plt
from nltk.stem import WordNetLemmatizer, PorterStemmer
# from sklearn.feature_extraction.text import TfidfVectorizer
# dler = nltk.downloader.Downloader()
# dler._update_index()
nltk.data.path.append('/kaggle/output')
nltk.download('stopwords')
nltk.download('wordnet')

from nltk.corpus import stopwords
from nltk.corpus import  wordnet

import warnings
warnings.filterwarnings("ignore")

In [ ]:
data_df = pd.read_csv('/kaggle/input/thesis-data/Nebuli_data.csv',index_col=0)
data_df.head()


In [ ]:
len(data_df.id.unique())

In [ ]:
data_df.isnull().sum()

In [ ]:
data_df = data_df[data_df['fulltext'].isnull()==False]
len(data_df[data_df.isnull()==False])

In [ ]:
data_df.isnull().sum()

In [ ]:
k = data_df.groupby(['parent_title']).agg({'id':'count'}).reset_index()
fig = px.bar(k, y="parent_title",x='id')
fig.show()

In [ ]:
data_df.loc[data_df['parent_title'] == 'Channels', 'parent_title'] = data_df['class_2']
data_df.loc[data_df['cat_parent_id'] == '2096', 'cat_parent_id'] = data_df['cat_id']
data_df.parent_title.unique(),len(data_df.parent_title.unique())

In [ ]:
k = data_df.groupby(['parent_title']).agg({'id':'count'}).reset_index()
k = k[k['id']>50]
fig = px.bar(k, y="parent_title",x='id')
data_df = data_df[data_df['parent_title'].isin(k.parent_title.unique())]
fig.show()

In [ ]:
data_df.parent_title.unique(),len(data_df.parent_title.unique())

In [ ]:
def preprocess_text(text):
    # Tokenization
    text= ''.join(e for e in text if e.isalnum() or e in {' ','\n'})
    tokens = nltk.word_tokenize(text.lower())

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words]

    # Lemmatization
#     lemmatizer = WordNetLemmatizer()
#     tokens = [lemmatizer.lemmatize(token) for token in tokens]
    tokens = [PorterStemmer().stem(token) for token in tokens]
    # Join the tokens back into a single string
    preprocessed_text = ' '.join(tokens)
    
    return preprocessed_text

data_df['title_cleaned'] = data_df['title'].apply(preprocess_text)
data_df['introtext'] = data_df['introtext'].astype(str)
data_df['introtext_cleaned'] =data_df['introtext'].apply(preprocess_text)
data_df['fulltext'] = data_df['fulltext'].astype(str)
data_df['fulltext_cleaned'] =data_df['fulltext'].apply(preprocess_text)
data_df.to_csv('preprocessed_nlp_data.csv')
data_df.head()

In [ ]:
data_df = data_df.reset_index(drop=True)
len(data_df)

In [ ]:
nlp_df = pd.read_csv('/kaggle/input/thesis-data/preprocessed_nlp_data.csv',index_col=0)
nlp_df.head() 

In [ ]:
feature_names_list = []
bag_nlp_df = data_df.copy(deep=True)
vectorizer = CountVectorizer()
bag_of_words = vectorizer.fit_transform(bag_nlp_df['fulltext_cleaned'].values.astype('U'))
feature_names = vectorizer.get_feature_names_out()
bow_df = pd.DataFrame(bag_of_words.toarray(), columns=feature_names)
bag_nlp_df = pd.concat([bag_nlp_df, bow_df], axis=1)
feature_names_list.extend(feature_names)
bag_nlp_df.tail()

In [ ]:
len(bow_df)

In [ ]:
df_bow_sklearn = pd.DataFrame(bag_of_words.toarray(),columns=feature_names)

In [ ]:
print(max(df_bow_sklearn.astype('bool').sum()))
series = df_bow_sklearn.astype('bool').sum(axis=0)
plt.hist(series)

In [ ]:
df_bow_sklearn = df_bow_sklearn.loc[:, (df_bow_sklearn.astype('bool').sum() >= 900) & (df_bow_sklearn.astype('bool').sum() <= 8000)]
df_bow_sklearn = df_bow_sklearn.loc[:, ~df_bow_sklearn.columns.str.contains(r'\d')]
series = df_bow_sklearn.astype('bool').sum(axis=0)
plt.hist(series)

In [ ]:
df_bow_sklearn.shape,bag_nlp_df.shape

In [ ]:
df_bow_sklearn.head()

In [ ]:
bag_nlp_df = pd.concat([data_df[['cat_id','cat_parent_id','grandparent_id']], df_bow_sklearn], axis=1)
bag_nlp_df.shape

In [ ]:
bag_nlp_df.tail()

In [ ]:
bag_nlp_df.to_csv('temp.csv')

In [ ]:
bag_nlp_df.isnull().sum()

In [ ]:
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
X = bag_nlp_df
wcss = []
K_range = range(1, 11)
for i in K_range:
    kmeans = KMeans(n_clusters=i, init='k-means++', random_state=42)
    kmeans.fit(X)
    wcss.append(kmeans.inertia_)
    
plt.figure(figsize=(10,5))
plt.plot(K_range, wcss, marker='o')
plt.title('Elbow Method')
plt.xlabel('Number of clusters')
plt.ylabel('WCSS')
plt.show()

In [ ]:
import numpy as np
import plotly.express as px
from sklearn import datasets
from sklearn.cluster import KMeans, SpectralClustering, AgglomerativeClustering, DBSCAN
import hdbscan

n_samples = 1500
random_state = 170
X = bag_nlp_df

methods = {
    "KMeans": KMeans(n_clusters=2),
    "SpectralClustering": SpectralClustering(n_clusters=2, affinity='nearest_neighbors', assign_labels='kmeans'),
    "Hierarchical": AgglomerativeClustering(n_clusters=10),
    "DBSCAN": DBSCAN(eps=0.1),
    "HDBSCAN": hdbscan.HDBSCAN(min_cluster_size=2)
}

results = {}

for name, method in methods.items():
    results[name] = method.fit_predict(X)
    
tsne = TSNE(n_components=2, perplexity=30, random_state=42)
reduced_data = tsne.fit_transform(X)

for name, labels in results.items():
    fig = px.scatter(reduced_data, color=labels, title=name)
    fig.show()


In [ ]:
reduced_data

In [ ]:
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score
silhouette = silhouette_score(reduced_data, results['KMeans'])
davies_bouldin = davies_bouldin_score(reduced_data,results['KMeans'])
calinski_harabasz = calinski_harabasz_score(reduced_data, results['KMeans'])

silhouette,davies_bouldin, calinski_harabasz

In [ ]:
silhouette = silhouette_score(reduced_data, results['SpectralClustering'])
davies_bouldin = davies_bouldin_score(reduced_data,results['SpectralClustering'])
calinski_harabasz = calinski_harabasz_score(reduced_data, results['SpectralClustering'])

silhouette,davies_bouldin, calinski_harabasz

In [ ]:
silhouette = silhouette_score(reduced_data, results['Hierarchical'])
davies_bouldin = davies_bouldin_score(reduced_data,results['Hierarchical'])
calinski_harabasz = calinski_harabasz_score(reduced_data, results['Hierarchical'])

silhouette,davies_bouldin, calinski_harabasz

In [ ]:
#silhouette = silhouette_score(reduced_data, results['DBSCAN'])
#davies_bouldin = davies_bouldin_score(reduced_data,results['DBSCAN'])
#calinski_harabasz = calinski_harabasz_score(reduced_data, results['DBSCAN'])

#silhouette,davies_bouldin, calinski_harabasz

In [ ]:
silhouette = silhouette_score(reduced_data, results['HDBSCAN'])
davies_bouldin = davies_bouldin_score(reduced_data,results['HDBSCAN'])
calinski_harabasz = calinski_harabasz_score(reduced_data, results['HDBSCAN'])

silhouette,davies_bouldin, calinski_harabasz

In [ ]:
results

In [ ]:
len(feature_names_list)

In [ ]:
import matplotlib.pyplot as plt
from wordcloud import WordCloud
word_cloud =WordCloud().generate(' '.join(feature_names_list))
plt.imshow(word_cloud)
plt.axis("off")
plt.show()

In [ ]:
data_df.columns

In [ ]:
bag_nlp_df

In [ ]:
bag_nlp_df['kmeans']=results['Hierarchical']
bag_nlp_df.head()

In [ ]:
bag_nlp_df.to_csv('final_data.csv')

In [ ]:
from scipy.cluster.hierarchy import dendrogram, linkage
bag_nlp_df_temp = bag_nlp_df.copy(deep=True)
linked = linkage(bag_nlp_df_temp,'complete')

plt.figure(figsize=(100, 50))
dendrogram(linked)
plt.title('Dendrogram')
plt.xlabel('Data Points')
plt.ylabel('Euclidean Distance')
plt.show()

In [ ]:
# import pandas as pd
# import dash_bio

# dash_bio.Clustergram(
#     data=bag_nlp_df,
#     column_labels=list(bag_nlp_df.columns.values),
#     row_labels=list(bag_nlp_df.index),
#     display_ratio=[0.1, 0.7]
# )

In [ ]:
temp = bag_nlp_df.drop(columns=['cat_id', 'level','cat_parent_id','grandparent_id'])
grouped = temp.groupby('kmeans').sum()
for cluster, row in grouped.iterrows():
    words = {}
    for word, value in row.iteritems():
        words[word] = value
    
    wordcloud = WordCloud(background_color='white').generate_from_frequencies(words)
    plt.figure()
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis('off')
    plt.title(f"Cluster {cluster}")
    plt.show()

In [ ]:
bag_nlp_df = pd.concat([data_df[['cat_id','parent_title','grandparent_id']], df_bow_sklearn], axis=1)
bag_nlp_df.shape

In [ ]:
temp = bag_nlp_df.drop(columns=['grandparent_id','cat_id'])
grouped = temp.groupby('parent_title').sum()
for cluster, row in grouped.iterrows():
    words = {}
    for word, value in row.iteritems():
        words[word] = value
    
    wordcloud = WordCloud(background_color='white').generate_from_frequencies(words)
    plt.figure()
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis('off')
    plt.title(f"{cluster} word cloud")
    plt.show()




In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from sklearn import datasets
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_curve, roc_auc_score
from sklearn.preprocessing import LabelBinarizer

# Assuming data is loaded as `df` and target column is 'target'
# Convert the target to string if they're not
bag_nlp_df['parent_title'] = bag_nlp_df['parent_title'].astype(str)

y = bag_nlp_df['parent_title']
X = bag_nlp_df.drop(columns=['parent_title','cat_id', 'grandparent_id'])

# Count the instances of each class
class_counts = y.value_counts()

# Find the classes that have at least 2 instances
valid_classes = class_counts[class_counts >= 2].index

# Filter the data to only include these classes
y = y[y.isin(valid_classes)]
X = X.loc[y.index]

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,stratify=y)

# Binarize target for ROC
lb = LabelBinarizer()
y_train_bin = lb.fit_transform(y_train)
y_test_bin = lb.transform(y_test)
# Initialize classifiers
classifiers = [
    ("Logistic Regression", LogisticRegression(max_iter=1000)),
    ("Decision Tree", DecisionTreeClassifier()),
    ("Random Forest", RandomForestClassifier())
]

# Data Collection
metrics_data = []
roc_data = []

# Model Training, Prediction & Metric Calculation
for name, clf in classifiers:
    clf.fit(X_train, y_train)
    predictions = clf.predict(X_test)
    report = classification_report(y_test, predictions, output_dict=True, zero_division=1)
    
    # Collecting metrics data
    for target_class in list(set(y_test.values)):
        metrics_data.append((name, target_class, report[target_class]['precision'], report[target_class]['recall'], report[target_class]['f1-score']))
        
    # ROC and AUC for multi-class (One-vs-All approach)
    y_prob = clf.predict_proba(X_test)
    for idx, target_class in enumerate(list(set(y_test.values))):
        fpr, tpr, _ = roc_curve(y_test_bin[:, idx], y_prob[:, idx])
        auc = roc_auc_score(y_test_bin[:, idx], y_prob[:, idx])
        roc_data.append((name, target_class, fpr, tpr, auc))
        
# Convert results to DataFrame for visualization
df_metrics = pd.DataFrame(metrics_data, columns=["Classifier", "Class", "Precision", "Recall", "F1-Score"])

# Visualizations
# Bar charts for metrics
fig = px.bar(df_metrics, x="Classifier", y=["Precision", "Recall", "F1-Score"], color="Class", barmode="group", title="Classification Metrics Comparison")
fig.show()

# ROC Curves
fig = go.Figure()
for name, target_class, fpr, tpr, auc in roc_data:
    fig.add_trace(go.Scatter(x=fpr, y=tpr, name=f'{name} - Class {target_class} (AUC: {auc:.2f})'))
fig.add_trace(go.Scatter(x=[0, 1], y=[0, 1], mode='lines', name='Baseline', line=dict(dash='dash')))
fig.update_layout(title="ROC Curve", xaxis_title="False Positive Rate", yaxis_title="True Positive Rate", xaxis=dict(scaleanchor="y", scaleratio=1), yaxis=dict(scaleanchor="x", scaleratio=1))
fig.show()


In [ ]:
roc_data

In [ ]:
df_metrics

In [ ]:
len(y_prob),y_prob[0]

In [ ]:
results

In [ ]:
report

In [ ]:
metrics_data

In [ ]:
bag_nlp_df.parent_title.unique()

In [ ]:
X